In [16]:
import pandas as pd
from sqlalchemy import create_engine

In [17]:
'''
Create a mapping of df dtypes to mysql data types (not perfect, but close enough)
'''
def dtype_mapping():
    return {'object' : 'TEXT',
        'int64' : 'INTEGER',
        'float64' : 'FLOAT',
        'datetime64' : 'TEXT',
        'bool' : 'BOOLEAN',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}

In [18]:
'''
Create a sqlalchemy engine
'''
def postgresql_engine(user = 'kremlin', password = '123456', host = '127.0.0.1', port = '5432', database = 'retail'):
    engine = create_engine("postgresql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database))
    return engine

In [19]:
'''
Create a postgresql connection from sqlalchemy engine
'''
def postgresql_conn(engine):
    conn = engine.raw_connection()
    return conn


In [20]:
'''
Create sql input for table names and types
'''
def gen_tbl_cols_sql(df):
    dmap = dtype_mapping()
    sql = "pi_db_uid SERIAL PRIMARY KEY"
    df1 = df.rename(columns = {"" : "nocolname"})
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for hl in hdrs_list:
        sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql


In [21]:
'''
Create a postgresql table from a df
'''
def create_postgresql_tbl_schema(df, conn, db, tbl_name):
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "CREATE TABLE {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()

In [22]:
'''
Write df data to newly create postgresql table
'''
def df_to_postgresql(df, engine, tbl_name):
    df.to_sql(tbl_name, engine, if_exists='replace')

In [24]:
infile = './data/customer'
db = 'retail'
db_tbl_name = 'customer'

df = pd.read_csv(infile,sep='|', header=None, names=["customer_id","customer_fname","customer_lname","customer_email","customer_password","customer_street","customer_city","customer_state","customer_zipcode"])
create_postgresql_tbl_schema(df, postgresql_conn(postgresql_engine()), db, db_tbl_name)
df_to_postgresql(df, postgresql_engine(), db_tbl_name)

In [25]:
infile = './data/orders'
db_tbl_name = 'orders'

df = pd.read_csv(infile,sep='|', header=None, names=["order_id","order_date","order_customer_id","order_status"])
create_postgresql_tbl_schema(df, postgresql_conn(postgresql_engine()), db, db_tbl_name)
df_to_postgresql(df, postgresql_engine(), db_tbl_name)

In [26]:
infile = './data/order_items'
db_tbl_name = 'order_items'

df = pd.read_csv(infile,sep='|', header=None, names=["order_item_id","order_item_order_id","order_item_product_id","order_item_quantity","order_item_subtotal","order_item_product_price"])
create_postgresql_tbl_schema(df, postgresql_conn(postgresql_engine()), db, db_tbl_name)
df_to_postgresql(df, postgresql_engine(), db_tbl_name)

In [27]:
infile = './data/departments'
db_tbl_name = 'departments'

df = pd.read_csv(infile,sep='|', header=None, names=['department_id', 'department_name'])
create_postgresql_tbl_schema(df, postgresql_conn(postgresql_engine()), db, db_tbl_name)
df_to_postgresql(df, postgresql_engine(), db_tbl_name)

In [28]:
infile = './data/categories'
db_tbl_name = 'categories'

df = pd.read_csv(infile,sep='|', header=None, names=["category_id", "category_department_id", "category_name"])
create_postgresql_tbl_schema(df, postgresql_conn(postgresql_engine()), db, db_tbl_name)
df_to_postgresql(df, postgresql_engine(), db_tbl_name)

In [29]:
infile = './data/products'
db_tbl_name = 'products'

df = pd.read_csv(infile,sep='|', header=None, names=["product_id","product_category_id","product_name","product_description","product_price","product_image"])
create_postgresql_tbl_schema(df, postgresql_conn(postgresql_engine()), db, db_tbl_name)
df_to_postgresql(df, postgresql_engine(), db_tbl_name)